# Lezione 37 — Introduzione al Deep Learning (concettuale)

**Obiettivi della lezione**
- Comprendere il neurone artificiale e il meccanismo forward/backward pass.
- Confrontare deep learning con machine learning classico per un data analyst.
- Identificare quando il deep learning è (e non è) la scelta giusta.
- Evidenziare limiti pratici per chi fa analisi dati (costi, dati, interpretabilità).

## Teoria concettuale approfondita

**Neurone artificiale (percettrone moderno)**
- Input $x$, pesi $w$, bias $b$. Output: $y = \sigma(w \cdot x + b)$ con funzione di attivazione non lineare $\sigma$.
- Funzione di attivazione tipiche: sigmoid (0-1, rischio saturazione), ReLU (0, max(0,z), semplice e stabile), tanh (-1,1).

**Forward pass**
- Calcolo diretto dell'output dato l'input: passa attraverso strati densi o convoluzionali; ogni strato applica trasformazioni affini + non linearità.

**Backward pass (backpropagation)**
- Calcolo dei gradienti della loss rispetto ai pesi usando la regola della catena.
- Aggiornamento pesi con ottimizzatore (es. SGD): $w \leftarrow w - \eta \cdot \nabla_w \mathcal{L}$.

**Differenze con ML classico (per un data analyst)**
- **Feature engineering**: nel deep learning è in gran parte automatizzata; nel ML classico è cruciale e manuale.
- **Dati richiesti**: deep learning richiede molti esempi e dati etichettati; ML classico può funzionare con meno dati.
- **Interpretabilità**: modelli profondi sono spesso black-box; modelli lineari/alberi sono più leggibili.
- **Costi computazionali**: reti profonde richiedono GPU/TPU; ML classico è più leggero.

**Perché NON è sempre la soluzione giusta**
- Scarso volume di dati o etichette rumorose.
- Vincoli di spiegabilità (audit, compliance).
- Vincoli di latenza o budget computazionale limitato.

**Limiti pratici per un data analyst**
- Curva di apprendimento degli strumenti (framework, tuning).
- Necessità di pipeline dati stabili, logging sperimentale, monitoraggio drift.
- Rischio di overfitting senza adeguata regolarizzazione e validazione.

## Schema mentale / mappa logica
- **Quando usare**: immagini, audio, testo lungo, pattern complessi dove feature manuali sono difficili; grandi volumi di dati; disponibilità di GPU e tempo.
- **Quando NON usare**: dataset piccolo, alta richiesta di interpretabilità, problemi tabellari semplici risolvibili con alberi/boosting, forti vincoli di latenza/budget.
- **Segnali pratici nei dati**: alta dimensionalità grezza (pixel, token), segnali non lineari, pattern locali ripetuti (immagini), dipendenze sequenziali (testo/tempo).
- **Pattern operativo**: partire da modello semplice (baseline), valutare se la complessità del problema giustifica una rete; monitorare overfitting; mantenere baseline classica per confronto.

## Notebook dimostrativo (concettuale, senza training complesso)
Mostriamo un percettrone multistrato minimale su un dataset toy 2D per classificazione lineare vs non lineare. Useremo solo NumPy per evidenziare forward e backward pass semplificati (1 hidden layer).

In [ ]:
# Importiamo NumPy per il calcolo numerico
import numpy as np
np.random.seed(42)  # fissiamo il seed per riproducibilità

In [ ]:
# Creiamo un piccolo dataset 2D non lineare (due moon) per mostrare il ruolo della non linearità
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=300, noise=0.1, random_state=42)
X[:5], y[:5]

In [ ]:
# Inizializziamo un MLP minimale: input -> hidden (ReLU) -> output (sigmoid)
input_dim = 2
hidden_dim = 8
output_dim = 1

# Pesiamo piccoli valori random per evitare saturazione iniziale
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)
b1 = np.zeros((1, hidden_dim))
W2 = 0.1 * np.random.randn(hidden_dim, output_dim)
b2 = np.zeros((1, output_dim))

# Funzioni di attivazione e derivate

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
# Definiamo forward pass: calcolo delle attivazioni intermedie e output

def forward(X_batch):
    z1 = X_batch @ W1 + b1           # combinazione lineare primo strato
    a1 = relu(z1)                    # non linearità
    z2 = a1 @ W2 + b2                # logits finali
    a2 = sigmoid(z2)                 # probabilità classe 1
    cache = {"X": X_batch, "z1": z1, "a1": a1, "z2": z2, "a2": a2}
    return a2, cache

In [ ]:
# Definiamo la loss binaria (log loss) e la backward pass per aggiornare i pesi

def compute_loss(y_true, y_pred):
    eps = 1e-8  # evitiamo log(0)
    return -np.mean(y_true * np.log(y_pred + eps) + (1 - y_true) * np.log(1 - y_pred + eps))


def backward(cache, y_true, y_pred, lr=0.1):
    global W1, b1, W2, b2
    m = y_true.shape[0]

    # Gradienti strato output
    dz2 = y_pred - y_true.reshape(-1, 1)         # derivata della log-loss wrt z2
    dW2 = cache["a1"].T @ dz2 / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    # Gradienti strato hidden
    da1 = dz2 @ W2.T
    dz1 = da1 * relu_derivative(cache["z1"])
    dW1 = cache["X"].T @ dz1 / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    # Aggiornamento pesi
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2

In [ ]:
# Eseguiamo un training brevissimo (poche epoche) solo per mostrare la dinamica

y = y.astype(float)
loss_history = []
for epoch in range(50):  # numero piccolo per dimostrazione
    y_pred, cache = forward(X)
    loss = compute_loss(y, y_pred)
    loss_history.append(loss)
    backward(cache, y, y_pred, lr=0.5)

loss_history[:5], loss_history[-5:]

In [ ]:
# Calcoliamo l'accuratezza finale per capire se la rete ha imparato qualcosa

y_pred_label = (y_pred.flatten() > 0.5).astype(int)
accuracy = np.mean(y_pred_label == y)
loss_history[-1], accuracy

### Osservazioni sul risultato
- Con poche epoche e un MLP minimo la loss scende e l'accuratezza cresce, mostrando la capacità di apprendere pattern non lineari.
- Non abbiamo fatto tuning serio: il focus è capire forward/backward e non ottenere SOTA.
- Limiti: nessuna validazione separata, rischio overfitting, nessuna regolarizzazione; serve baseline classica per confronto.

## Esercizi svolti (step-by-step)
Gli esercizi servono a collegare concetti e pratica minimale, senza entrare in training complesso.

In [ ]:
# Esercizio 1: variazione ampiezza hidden layer e osservazione della loss

for hidden_dim in [4, 8, 16]:
    # reinizializziamo pesi per confronto equo
    W1 = 0.1 * np.random.randn(input_dim, hidden_dim)
    b1 = np.zeros((1, hidden_dim))
    W2 = 0.1 * np.random.randn(hidden_dim, output_dim)
    b2 = np.zeros((1, output_dim))

    loss_history = []
    for epoch in range(40):
        y_pred, cache = forward(X)
        loss = compute_loss(y, y_pred)
        loss_history.append(loss)
        backward(cache, y, y_pred, lr=0.4)

    print(f"hidden={hidden_dim}, loss finale={loss_history[-1]:.3f}")

In [ ]:
# Esercizio 2: proviamo una funzione di attivazione diversa (tanh) e confrontiamo

def tanh(z):
    return np.tanh(z)

def tanh_derivative(z):
    return 1 - np.tanh(z) ** 2

# Riutilizziamo lo stesso schema ma sostituendo ReLU con tanh
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)
b1 = np.zeros((1, hidden_dim))
W2 = 0.1 * np.random.randn(hidden_dim, output_dim)
b2 = np.zeros((1, output_dim))


def forward_tanh(X_batch):
    z1 = X_batch @ W1 + b1
    a1 = tanh(z1)
    z2 = a1 @ W2 + b2
    a2 = sigmoid(z2)
    cache = {"X": X_batch, "z1": z1, "a1": a1, "z2": z2, "a2": a2}
    return a2, cache


def backward_tanh(cache, y_true, y_pred, lr=0.1):
    global W1, b1, W2, b2
    m = y_true.shape[0]
    dz2 = y_pred - y_true.reshape(-1, 1)
    dW2 = cache["a1"].T @ dz2 / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m
    da1 = dz2 @ W2.T
    dz1 = da1 * tanh_derivative(cache["z1"])
    dW1 = cache["X"].T @ dz1 / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2

# Training breve con tanh
loss_history_tanh = []
for epoch in range(40):
    y_pred, cache = forward_tanh(X)
    loss = compute_loss(y, y_pred)
    loss_history_tanh.append(loss)
    backward_tanh(cache, y, y_pred, lr=0.3)

loss_history_tanh[-1]

In [ ]:
# Esercizio 3: confronto con baseline lineare (senza hidden) per vedere il limite della linearità

# Modello lineare: logistic regression manuale (1 strato)
W_lin = 0.1 * np.random.randn(input_dim, 1)
b_lin = np.zeros((1, 1))


def forward_linear(X_batch):
    z = X_batch @ W_lin + b_lin
    a = sigmoid(z)
    return a, {"X": X_batch, "z": z, "a": a}


def backward_linear(cache, y_true, y_pred, lr=0.1):
    global W_lin, b_lin
    m = y_true.shape[0]
    dz = y_pred - y_true.reshape(-1, 1)
    dW = cache["X"].T @ dz / m
    db = np.sum(dz, axis=0, keepdims=True) / m
    W_lin -= lr * dW
    b_lin -= lr * db

# Training breve per baseline
loss_lin_hist = []
for epoch in range(40):
    y_pred_lin, cache_lin = forward_linear(X)
    loss_lin = compute_loss(y, y_pred_lin)
    loss_lin_hist.append(loss_lin)
    backward_linear(cache_lin, y, y_pred_lin, lr=0.3)

loss_lin_hist[-1]

## Conclusione operativa
- Portarsi a casa: il deep learning automatizza parte del feature learning ma richiede dati, calcolo e cura ingegneristica. Il forward/backward pass sono il cuore dell’ottimizzazione.
- Errori da evitare: scegliere reti complesse senza baseline, ignorare overfitting/validazione, sottovalutare interpretabilità e costi.
- Ponte verso la prossima lezione: introdurremo i modelli generativi e la differenza tra predizione e generazione (LLM come modelli probabilistici).